In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings.local_ventura")

'config.settings.local_ventura'

In [3]:
import django
os.chdir('../')
os.getcwd()
from django_pandas.io import read_frame
import plotly_express as px
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
django.setup()

In [13]:
import plotly
import chart_studio.plotly as py
from plotly import tools
import plotly.graph_objs as go

In [5]:
from chapters.models import Chapter
from django.db import models

In [3]:
test = Chapter.objects.first()

In [4]:
test.actives().count()

29

In [6]:
test.members.exclude(models.Q(depledge__isnull=True)).first().depledge.reason

'volunteer'

In [8]:
colors = {
    'actives': 'darkred',
    'pledges': 'gold',
    'events': 'green',
    'submissions': 'yellow',
    'Bro': 'darkred',
    'Ops': 'skyblue',
    'Pro': 'gold',
    'Ser': 'magenta',
    'volunteer': 'darkred',
    'time': 'skyblue',
    'grades': 'gold',
    'interest': 'magenta',
    'vote': 'orange',
    'withdrew': 'grey',
    'transfer': 'turquoise',
    'other': 'turquoise',
}

In [9]:
from regions.models import Region

In [10]:
test_region = Region.objects.last()

In [11]:
test_region.chapters.all()

<QuerySet [<Chapter: Epsilon>, <Chapter: Lambda Delta>, <Chapter: Mu Delta>, <Chapter: Omicron Gamma>, <Chapter: Rho Delta>, <Chapter: SCU Colony>, <Chapter: SJSU Colony>, <Chapter: UW Colony>]>

In [14]:
DataOut=[]
legend=True
for chapter in Chapter.objects.all():
    if chapter.name == "TEST" or not chapter.active:
        continue
    trace= go.Bar(
        x=[chapter.name],
        y=[chapter.actives().count()],
        visible=True,
        name=f"Actives",
        marker=dict(color=colors['actives']),
        showlegend=legend,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[chapter.name],
        y=[chapter.pledges().count()],
        visible=True,
        name=f"Pledges",
        marker=dict(color=colors['pledges']),
        showlegend=legend,
    )
    legend=False
    DataOut.append(trace)

In [15]:
menu = [
    dict(label = "ALL",
         method = 'update',
         args = [{'visible': [True for data_obj in DataOut]},
                 ]),
       ]
for region in Region.objects.all().order_by('name'):
    if region.name == "Test":
        continue
    chapter_names = region.chapters.all().values_list('name', flat=True)
    menu_obj = dict(label = region.name,
         method = 'update',
         args = [{'visible': [True if data_obj['x'][0] in chapter_names else False for data_obj in DataOut]},
                 ])
    menu.append(menu_obj)
updatemenus = list([
    dict(active=0,
         buttons=menu,
    )
])

In [16]:
layout = go.Layout(
    barmode='stack',
    updatemenus=updatemenus,
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="chapter_activeVSpledges.html")

'chapter_activeVSpledges.html'

In [17]:
from scores.models import ScoreType

In [37]:
DataOut=[]
legend=True
for chapter in Chapter.objects.all():
    if chapter.name == "TEST" or not chapter.active:
        continue
    for evt_type in [x.value[0] for x in ScoreType.SECTION]:
        count = chapter.events.filter(
            date__lte=models.BIENNIUM_DATES['Fall 2018']['end'],
            date__gte=models.BIENNIUM_DATES['Fall 2018']['start']
        ).filter(type__section=evt_type).count()
        trace= go.Bar(
            x=[chapter.name],
            y=[count],
            visible=True,
            name=f"{evt_type}",
            marker=dict(color=colors[evt_type]),
            showlegend=legend,
        )
        DataOut.append(trace)
    legend=False
layout = go.Layout(
    barmode='stack',
    updatemenus=updatemenus,
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="chapter_eventTypes_fall2018.html")

'chapter_eventTypes_fall2018.html'

In [19]:
from forms.models import Depledge

In [20]:
DataOut=[]
legend=True
for chapter in Chapter.objects.all():
    if chapter.name == "TEST" or not chapter.active:
        continue
    for type_value in [x.value[0] for x in Depledge.REASONS]:
        count = chapter.members.exclude(models.Q(depledge__isnull=True)).filter(depledge__reason=type_value).count()
        trace= go.Bar(
            x=[chapter.name],
            y=[count],
            visible=True,
            name=f"{type_value}",
            marker=dict(color=colors[type_value]),
            showlegend=legend,
        )
        DataOut.append(trace)
    legend = False
layout = go.Layout(
    barmode='stack',
    updatemenus=updatemenus,
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="chapter_depledges.html")

'chapter_depledges.html'

In [21]:
from core import models

In [22]:
models.BIENNIUM_DATES

{'Fall 2018': {'start': datetime.datetime(2018, 7, 1, 0, 0),
  'end': datetime.datetime(2018, 12, 31, 0, 0)},
 'Spring 2019': {'start': datetime.datetime(2019, 1, 1, 0, 0),
  'end': datetime.datetime(2019, 6, 30, 0, 0)},
 'Fall 2019': {'start': datetime.datetime(2019, 7, 1, 0, 0),
  'end': datetime.datetime(2019, 12, 31, 0, 0)},
 'Spring 2020': {'start': datetime.datetime(2020, 1, 1, 0, 0),
  'end': datetime.datetime(2020, 6, 30, 0, 0)}}

In [23]:
from events.models import Event

In [24]:
events = Event.objects.filter(
    date__lte=models.BIENNIUM_DATES['Fall 2019']['end'],
    date__gte=models.BIENNIUM_DATES['Fall 2019']['start'])

In [25]:
from forms.models import Initiation, StatusChange
from tasks.models import TaskChapter
from users.models import UserRoleChange

In [32]:
DataOut=[]

In [33]:
for i, (date_name, start_end)  in enumerate(models.BIENNIUM_DATES.items()):
    trace= go.Bar(
        x=[f"Events"],
        y=[Event.objects.filter(
            date__lte=start_end['end'],
            date__gte=start_end['start']).count()],
        visible=True,
        name=f"Events {date_name}",
        marker=dict(color='darkred'),
        showlegend=True,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Initiations"],
        y=[Initiation.objects.filter(
            created__lte=start_end['end'],
            created__gte=start_end['start']).count()],
        visible=True,
        name=f"Initiations {date_name}",
        marker=dict(color='gold'),
        showlegend=True,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Status Change"],
        y=[StatusChange.objects.filter(
            created__lte=start_end['end'],
            created__gte=start_end['start']).count()],
        visible=True,
        name=f"Status Change {date_name}",
        marker=dict(color='magenta'),
        showlegend=True,
    )
    DataOut.append(trace)
    trace= go.Bar(
        x=[f"Officer Reports"],
        y=[UserRoleChange.objects.filter(
            start__lte=start_end['end'],
            start__gte=start_end['start']).count()],
        visible=True,
        name=f"Officer Reports {date_name}",
        marker=dict(color='cyan'),
        showlegend=True,
    )
    DataOut.append(trace)
layout = go.Layout(
)
fig = go.Figure(data=DataOut, layout=layout)
plotly.offline.plot(fig, filename="Counts.html")

'Counts.html'

In [27]:
for i, (info, test)  in enumerate(models.BIENNIUM_DATES.items()):
    print(i, info, test)

0 Fall 2018 {'start': datetime.datetime(2018, 7, 1, 0, 0), 'end': datetime.datetime(2018, 12, 31, 0, 0)}
1 Spring 2019 {'start': datetime.datetime(2019, 1, 1, 0, 0), 'end': datetime.datetime(2019, 6, 30, 0, 0)}
2 Fall 2019 {'start': datetime.datetime(2019, 7, 1, 0, 0), 'end': datetime.datetime(2019, 12, 31, 0, 0)}
3 Spring 2020 {'start': datetime.datetime(2020, 1, 1, 0, 0), 'end': datetime.datetime(2020, 6, 30, 0, 0)}
